In [ ]:
import torch

In [ ]:
phonies = {}

In [ ]:
phonies

{}

In [ ]:
def get_phony(device, requires_grad):
    key = (device, requires_grad)
    
    try:
        # if already exist
        phony = phonies[key]
    except KeyError:
        phony = torch.empty(0, requires_grad=requires_grad)
        phonies[key] = phony
    
    return phony

In [ ]:
device = torch.cuda.device("cuda:0")

In [ ]:
get_phony(device, True)

tensor([], requires_grad=True)

In [ ]:
phonies[(device, True)]

tensor([], requires_grad=True)

In [ ]:
class Fork(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        phony = get_phony(input.device, requires_grad=False)
        return input.detach(), phony.detach()

    @staticmethod
    def backward(ctx, grad_input, grad_grad):
        return grad_input

In [ ]:
class Join(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, phony):
        return input.detach()
    
    @staticmethod
    def backward(ctx, grad_input):
        return grad_input, None

In [ ]:
def fork(input):
    if torch.is_grad_enabled() and input.requires_grad:
        input, phony = Fork.apply(input)
    else:
        phony = get_phony(input.device, requires_grad=False)
    return input, phony

In [ ]:
def join(input, phony):
    if torch.is_grad_enabled() and (input.requires_grad or phony.requires_grad):
        input = Join.apply(input, phony)
    return input

In [ ]:
def depend(fork_from, join_to):
    fork_from, phony = fork(fork_from)
    join_to = join(join_to, phony)

In [ ]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = torch.tensor([4., 5., 6.], requires_grad=True)

In [ ]:
depend(x, y)

In [ ]:
y.sum().backward()

In [ ]:
y.grad

tensor([1., 1., 1.])

In [ ]:
x.grad

In [ ]:
class Operation(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input + 1
    
    @staticmethod
    def backward(ctx, grad_input):
        return grad_input, None

In [ ]:
batch_1 = torch.randn(2, 3, requires_grad=True)

In [ ]:
batch_1.grad

In [ ]:
output = Operation.apply(batch_1)

In [ ]:
output.sum().backward()